## load library

In [60]:
#python

import json
import copy
import random
import yaml

from datetime import datetime
from typing import Any, Dict, Tuple, Union, List
from tqdm import tqdm,notebook

#automl optuna
import optuna

#sklearn

from sklearn.metrics import f1_score

#numpy

import numpy as np

#pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models

#baseline

import warnings

warnings.filterwarnings('ignore')

import os
import sys

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('/opt/ml/code/src/'))))

from src.dataloader import create_dataloader
from src.loss import CustomCriterion
from src.model import Model
from src.trainer import TorchTrainer
from src.utils.common import get_label_counts, read_yaml
from src.utils.macs import calc_macs
from src.utils.torch_utils import check_runtime, model_info
from src.augmentation.policies import simple_augment_test
from src.utils.inference_utils import run_model
from src.utils.torch_utils import save_model

## hyperparameter

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## fix seed

In [3]:
#torch seed
torch.manual_seed(30)
torch.cuda.manual_seed(30)

#numpy seed
np.random.seed(30)

#python seed
random.seed(30)

## load config

In [4]:
DATA_CONFIG = read_yaml(cfg="/opt/ml/code/configs/data/taco.yaml")

## define hyperparameter search space function

In [13]:
def search_hyperparam(trial: optuna.trial.Trial) -> Dict[str, Any]:
    """Search hyperparam from user-specified search space."""
    epochs = trial.suggest_int("epochs", low=50, high=200, step=50)
    img_size = trial.suggest_categorical("img_size", [18,24,30,36,42])
    #epochs = 200
    #img_size = 96
    n_select = trial.suggest_int("n_select", low=0, high=6, step=2)
    batch_size = trial.suggest_int("batch_size", low=16, high=64, step=16)
    return {
        "EPOCHS": epochs,
        "IMG_SIZE": img_size,
        "n_select": n_select,
        "BATCH_SIZE": batch_size
    }

## load pretrained model

In [69]:
model_instance = models.shufflenet_v2_x0_5(pretrained=True)

In [70]:
model_instance.fc = nn.Linear(in_features=1024, out_features=9, bias=True)

In [71]:
model_instance

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

## define train function

In [54]:
def train(model_instance, model_path, optimizer, scheduler, criterion, scaler, n_epoch, train_dl, val_dl, device):
    
    #n_epoch = 30

    best_test_acc = -1.0
    best_test_f1 = -1.0

    num_classes = 9

    label_list = [i for i in range(num_classes)]

    for epoch in range(n_epoch):
        running_loss, correct, total = 0.0, 0, 0
        preds, gt = [], []
        pbar = notebook.tqdm(enumerate(train_dl), total=len(train_dl))
        model_instance.train()
        for batch, (data, labels) in pbar:

            data, labels = data.to(device), labels.to(device)

            if scaler:
                with torch.cuda.amp.autocast():
                    outputs = model_instance(data)
            else:
                outputs = model_instance(data)
            outputs = torch.squeeze(outputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()

            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            scheduler.step()

            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            preds += pred.to("cpu").tolist()
            gt += labels.to("cpu").tolist()

            running_loss += loss.item()
            pbar.update()
            pbar.set_description(
                f"Train: [{epoch + 1:03d}] "
                f"Loss: {(running_loss / (batch + 1)):.3f}, "
                f"Acc: {(correct / total) * 100:.2f}% "
                f"F1(macro): {f1_score(y_true=gt, y_pred=preds, labels=label_list, average='macro', zero_division=0):.2f}"
            )
        pbar.close()

        _, test_f1, test_acc = test(
            model=model_instance, test_dataloader=val_dl, scaler=scaler, criterion=criterion
        )
        if best_test_f1 > test_f1:
            continue
        best_test_acc = test_acc
        best_test_f1 = test_f1
        print(f"Model saved. Current best test f1: {best_test_f1:.3f}")
        save_model(
            model=model_instance,
            path=model_path,
            data=data,
            device=device,
        )
    
    return best_test_acc,best_test_f1

In [55]:
@torch.no_grad()
def test(model,test_dataloader,scaler,criterion):
    """Test model.

    Args:
        test_dataloader: test data loader module which is a iterator that returns (data, labels)

    Returns:
        loss, f1, accuracy
    """

    #n_batch = _get_n_batch_from_dataloader(test_dataloader)

    running_loss = 0.0
    preds = []
    gt = []
    correct = 0
    total = 0

    num_classes = 9
    label_list = [i for i in range(num_classes)]

    pbar = notebook.tqdm(enumerate(test_dataloader), total=len(test_dataloader))
    model.to(device)
    model.eval()
    for batch, (data, labels) in pbar:
        data, labels = data.to(device), labels.to(device)

        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(data)
        else:
            outputs = model(data)
        outputs = torch.squeeze(outputs)
        running_loss += criterion(outputs, labels).item()

        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

        preds += pred.to("cpu").tolist()
        gt += labels.to("cpu").tolist()
        pbar.update()
        pbar.set_description(
            f" Val: {'':5} Loss: {(running_loss / (batch + 1)):.3f}, "
            f"Acc: {(correct / total) * 100:.2f}% "
            f"F1(macro): {f1_score(y_true=gt, y_pred=preds, labels=label_list, average='macro', zero_division=0):.2f}"
        )
    loss = running_loss / len(test_dataloader)
    accuracy = correct / total
    f1 = f1_score(
        y_true=gt, y_pred=preds, labels=label_list, average="macro", zero_division=0
    )
    
    return loss, f1, accuracy

## define objective function

In [73]:
def objective(trial: optuna.trial.Trial, device) -> Tuple[float, int, float]:
    
    global before_macs,success
    
    data_config = copy.deepcopy(DATA_CONFIG)
    model_instance = models.shufflenet_v2_x0_5(pretrained=True)
    model_instance.fc = nn.Linear(in_features=1024, out_features=9, bias=True)

    # hyperparams: EPOCHS, IMG_SIZE, n_select, BATCH_SIZE
    hyperparams = search_hyperparam(trial)

    data_config["AUG_TRAIN_PARAMS"]["n_select"] = hyperparams["n_select"]
    data_config["BATCH_SIZE"] = hyperparams["BATCH_SIZE"]
    data_config["EPOCHS"] = hyperparams["EPOCHS"]
    data_config["IMG_SIZE"] = hyperparams["IMG_SIZE"]

    log_dir = os.path.join("exp", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    

    macs = calc_macs(model_instance, (3, data_config["IMG_SIZE"], data_config["IMG_SIZE"]))
    print(f"macs: {macs}")
    
    if trial.number > 1:
        if success:
            if macs > before_macs:
                print(f' trial: {trial.number}, I want to search lower macs: now {macs} >>> before {before_macs}')
                raise optuna.structs.TrialPruned()
    
    if macs>2500000.0:      ########
        print(f' trial: {trial.number}, This model has very large macs:{macs}')
        success = False
        raise optuna.structs.TrialPruned()##############
    
    #training
    
    #fixed hyperparameter
    
    model_instance = torch.nn.DataParallel(model_instance) #half tensor를 float tensor로 바꿔줌
    
    train_path = os.path.join(data_config["DATA_PATH"], "train")
    
    train_dl, val_dl, test_dl = create_dataloader(data_config)
    
    criterion = CustomCriterion(
        samples_per_cls=get_label_counts(train_path)
        if data_config["DATASET"] == "TACO"
        else None,
        device=device,
        #loss_type="weighted"
        #loss_type="customloss"
        #loss_type="label_smoothing"
    )
    
    # Amp loss scaler
    scaler = (
        torch.cuda.amp.GradScaler() if data_config['FP16'] and device != torch.device("cpu") else None
    )
    #scaler=None
    
    # Create optimizer, scheduler, criterion
    optimizer = torch.optim.SGD(
        model_instance.parameters(), lr=0.1, momentum=0.9
    )
    
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=data_config["INIT_LR"],
        steps_per_epoch=len(train_dl),
        epochs=data_config["EPOCHS"],
        pct_start=0.05,
    )
    

    save_path = os.path.join(log_dir, "best.pt")
    
    print(f"train {data_config['EPOCHS']} epochs this model!")
    
    os.makedirs(log_dir, exist_ok=True)
    
    with open(os.path.join(log_dir, 'data.yml'), 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
    
    _, best_f1 = train(
        model_instance,
        save_path,
        optimizer,
        scheduler, 
        criterion, 
        scaler,
        data_config["EPOCHS"],
        train_dl, 
        val_dl, 
        device
    )
    
    before_macs = macs
    success = True

    return best_f1, macs

In [76]:
def tune(device, study_name= "pstage_automl"):

    sampler = optuna.samplers.MOTPESampler(n_startup_trials=20)

    study = optuna.create_study(
        directions=["maximize", "minimize"],
        study_name=study_name,
        sampler=sampler,
        load_if_exists=True
    )
    
    study.optimize(lambda trial: objective(trial, device), n_trials=100)

    pruned_trials = [
        t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED
    ]
    complete_trials = [
        t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE
    ]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trials:")
    best_trials = study.best_trials

    ## trials that satisfies Pareto Fronts
    for tr in best_trials:
        print(f"  value1:{tr.values[0]}, value2:{tr.values[1]}")
        for key, value in tr.params.items():
            print(f"    {key}:{value}")

    return study

In [63]:
study_name="pstage_automl"

study = tune(device, model_instance)

[I 2021-06-26 16:44:26,060] A new study created in memory with name: pstage_automl


macs: 650993.0
train 150 epochs this model!



Model saved. Current best test f1: 0.163


KeyboardInterrupt: 

In [72]:
study_name="pstage_automl2"

study = tune(device, model_instance, study_name=study_name)

[I 2021-06-26 19:28:24,469] A new study created in memory with name: pstage_automl2


macs: 2305721.0
train 50 epochs this model!



Model saved. Current best test f1: 0.158



Model saved. Current best test f1: 0.186



Model saved. Current best test f1: 0.188


...

[I 2021-06-27 05:33:28,309] Trial 1 finished with values: [0.41540978821084523, 703217.0] and parameters: {'epochs': 150, 'img_size': 24, 'n_select': 0, 'batch_size': 32}. 
[I 2021-06-27 05:33:28,352] Trial 2 pruned. 



macs: 839993.0
 trial: 2, I want to search lower macs: now 839993.0 >>> before 703217.0
macs: 703217.0
train 50 epochs this model!



Model saved. Current best test f1: 0.395



Model saved. Current best test f1: 0.397


KeyboardInterrupt: 

In [77]:
study_name="pstage_automl3"

study = tune(device, study_name=study_name)

[I 2021-06-27 07:08:06,152] A new study created in memory with name: pstage_automl3


macs: 2100689.0
train 150 epochs this model!



Model saved. Current best test f1: 0.228



Model saved. Current best test f1: 0.245



Model saved. Current best test f1: 0.262


...


Model saved. Current best test f1: 0.327



Model saved. Current best test f1: 0.329



Model saved. Current best test f1: 0.340



Model saved. Current best test f1: 0.342


KeyboardInterrupt: 